# Manual import of rlc json file from open data portal

https://www.postgresqltutorial.com/postgresql-python/query/

In [1]:
import os
from psycopg2 import connect
import configparser
import numpy as np

import requests
from psycopg2.extras import execute_values

import pandas as pd
import pandas.io.sql as pandasql

import csv

In [2]:
CONFIG=configparser.ConfigParser()
CONFIG.read('/home/cnangini/db.cfg')
dbset=CONFIG['DBSETTINGS']
conn=connect(**dbset)
i=0

# Functions

In [3]:
def pull_rlc(conn):
  '''
  Connect to bigdata RDS, pull Red Light Camera json file from Open Data API,
  and overwrite existing rlc table in the vz_safety_programs_staging schema.
  '''

  local_table='vz_safety_programs_staging.rlc_manual'

  url = "https://secure.toronto.ca/opendata/cart/red_light_cameras.json"
  return_json = requests.get(url).json()
  rows = [list(feature.values()) for feature in return_json]
  insert = 'INSERT INTO {0} VALUES %s'.format(local_table)
  with conn:
    with conn.cursor() as cur:
        print(cur)
        print(insert)
        print(rows)
        execute_values(cur, insert, rows)
        print(rows)

In [4]:
def pull_rlc_csv(conn):
  '''
  Connect to bigdata RDS, pull Red Light Camera json file from Open Data API,
  and overwrite existing rlc table in the vz_safety_programs_staging schema.
  '''

  local_table='vz_safety_programs_staging.rlc_fromcsv'

  url = "https://secure.toronto.ca/opendata/cart/red_light_cameras.csv"

  with requests.Session() as s:
    download = s.get(url)
    print(download)
    
    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        if row != ['RLC']:
            print(row)
            
    cur = conn.cursor()
    insert_query = "INSERT INTO users VALUES {}".format("(10, 'hello@dataquest.io', 'Some Name', '123 Fake St.')")
    cur.execute(insert_query)
    conn.commit()

# Truncate view, pull from open data, and check

In [5]:
# pull_rlc_csv(conn)

## Truncate first

In [6]:
query='''
TRUNCATE vz_safety_programs_staging.rlc_manual;
'''

cur = conn.cursor()
cur.execute(query)
cur.close()

In [7]:
# Check size

query='''
SELECT COUNT(*) FROM vz_safety_programs_staging.rlc_manual;
'''

cur = conn.cursor()
cur.execute(query)
row = cur.fetchone()
print(row)

cur.close()

(0,)


## Pull from open data

In [8]:
pull_rlc(conn)

<cursor object at 0x7fae6ad97ed0; closed: 0>
INSERT INTO vz_safety_programs_staging.rlc_manual VALUES %s
[['PARLIAMENT ST', -79.364023, None, None, 'Richmond St E And Parliament St', '6001', None, 4834744.176, '0223', None, 'Toronto and East York', 'RICHMOND ST E', None, None, '2007-11-09', 315752.39, None, 43.654559, None, 'Toronto Centre(13)', '51', None], ['YORK ST', -79.380868, None, None, 'Lake Shore Blvd W And York St', '6002', None, 4833286.82, '0205', None, 'Toronto and East York', 'LAKE SHORE BLVD W', None, None, '2007-11-09', 314395.743, None, 43.64146, 'WB LAKE SHORE BLVD', 'Spadina-Fort York(10)', '52', None], ['CARPENTER RD', -79.447589, None, None, 'Steeles Ave W And Carpenter Rd / Private Access @ Shopping Centre', '6003', None, 4850007.08, '1564', None, 'North York', 'STEELES AVE W', None, None, '2007-11-09', 309002.26, 'PRIVATE ACCESS', 43.792013, None, 'York Centre(6)', '32', 'SHOPPING CENTRE'], ['HILDA AVE', -79.429267, None, None, 'Steeles Ave W And Hilda Ave', '600

InvalidDatetimeFormat: invalid input syntax for type date: "0223"
LINE 1: ...d St E And Parliament St','6001',NULL,4834744.176,'0223',NUL...
                                                             ^


## Change owner

In [9]:
query='''
ALTER TABLE vz_safety_programs_staging.rlc_manual
    OWNER to vz_admins;
'''

cur = conn.cursor()
cur.execute(query)
cur.close()

## Count rows in updated table

In [10]:
query='''
SELECT COUNT(*) FROM vz_safety_programs_staging.rlc_manual;
'''

cur = conn.cursor()
cur.execute(query)
row = cur.fetchone()
print(row)

cur.close()

(148,)
